In [51]:
import pandas as pd
import json

In [52]:
import geopandas as gpd
from shapely.geometry import Point

# Your property data
property_metadata = pd.read_csv("../data/raw/properties.csv")

# Convert the coordinates column to a suitable format (assuming it's a string like '[lat, lon]')
property_metadata['coordinates'] = property_metadata['coordinates'].apply(eval)

# Create points and names from the property_metadata
coords = [Point(xy[1], xy[0]) for xy in property_metadata['coordinates']]
names = property_metadata['name'].tolist()

# Convert to GeoDataFrame
property = gpd.GeoDataFrame({'name': names, 'geometry': coords})


In [53]:
# Assuming property_gdf and victoria_gdf are already defined as per your provided code...
victoria_gdf = gpd.read_file('../data/Statistical_area_level2/victoria.geojson')

# Perform a spatial join
property_SA2 = gpd.sjoin(property, victoria_gdf, how="inner", op="within")

/home/jqqian1/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
/tmp/ipykernel_9036/2101387466.py:5: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  property_SA2 = gpd.sjoin(property, victoria_gdf, how="inner", op="within")


In [54]:
station = gpd.read_file('../data/external_SA2/Station.geojson')

In [55]:
station_SA2 = gpd.sjoin(station, victoria_gdf, how="inner", op="within")

/home/jqqian1/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [56]:
property_agg = pd.merge(property_SA2, station_SA2, on='SA2_CODE21', how='left')

In [57]:
merged = property_agg[['SA2_CODE21','geometry_x', 'geometry_y']]

In [58]:
merged.rename(columns={'geometry_x': 'property_geometry', 'geometry_y': 'station_geometry'}, inplace=True)

/tmp/ipykernel_9036/2385872553.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged.rename(columns={'geometry_x': 'property_geometry', 'geometry_y': 'station_geometry'}, inplace=True)


In [59]:
park = gpd.read_file('../data/external_SA2/Park.geojson')

In [60]:
park_SA2 = gpd.sjoin(park, victoria_gdf, how="inner", op="within")

/home/jqqian1/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [61]:
property_agg1 = pd.merge(merged, park_SA2, on='SA2_CODE21', how='left')

In [62]:
merged = property_agg1[['SA2_CODE21','property_geometry', 'station_geometry', 'geometry']]
merged.rename(columns={'geometry': 'park_geometry'}, inplace=True)

/tmp/ipykernel_9036/2976054265.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged.rename(columns={'geometry': 'park_geometry'}, inplace=True)


In [63]:
shop = gpd.read_file('../data/external_SA2/Mall, Shopping Centre & Department Store.geojson')

In [64]:
shop_SA2 = gpd.sjoin(shop, victoria_gdf, how="inner", op="within")

/home/jqqian1/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [65]:
property_agg2 = pd.merge(merged, shop_SA2, on='SA2_CODE21', how='left')

In [66]:
merged = property_agg2[['SA2_CODE21','property_geometry', 'station_geometry', 'park_geometry', 'geometry']]
merged.rename(columns={'geometry': 'shop_geometry'}, inplace=True)

/tmp/ipykernel_9036/256104151.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged.rename(columns={'geometry': 'shop_geometry'}, inplace=True)


In [67]:
merged

,SA2_CODE21,property_geometry,station_geometry,park_geometry,shop_geometry
0,206041503,POINT (144.96912 -37.80951),None,None,POINT (144.96643 -37.81279)
1,206041503,POINT (144.96912 -37.80951),None,None,POINT (144.96410 -37.81202)
2,206041503,POINT (144.97440 -37.81475),None,None,POINT (144.96643 -37.81279)
3,206041503,POINT (144.97440 -37.81475),None,None,POINT (144.96410 -37.81202)
4,206041503,POINT (144.96785 -37.81540),None,None,POINT (144.96643 -37.81279)
...,...,...,...,...,...
17280,212031555,POINT (145.37630 -38.08402),None,None,None
17281,212031555,POINT (145.34483 -38.07650),None,None,None
17282,212031555,POINT (145.36043 -38.08417),None,None,None
17283,212031555,POINT (145.35015 -38.08134),None,None,None
